In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Main
!ls

/content/drive/My Drive/Main
chest_xray_gen1000.h5  data_and_rcmodel.ipynb  generated_images    real_vs_fake.ipynb
chest_xray.h5	       gan_training.ipynb      generated_models    Tensorboard
classification_model   gan_training.py	       generate_img.ipynb  trained_net.png


In [ ]:

import pandas as pd
import numpy as np
import os
import h5py
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf

h5_path = '/content/drive/My Drive/Main/chest_xray.h5'

# Load images dataset using h5py
with h5py.File(h5_path, 'r') as h5_data:
    img_ds = h5_data['images'][:]
    img_ds = img_ds / 255
train_size = int(0.9 * len(img_ds))
x_train = img_ds[:train_size]
x_test = img_ds[train_size:]

from keras.models import Model,Sequential
from keras.layers import Dense,Flatten,Dropout,Activation,Lambda,Reshape
from keras.layers import Conv2D,ZeroPadding2D,UpSampling2D,Conv2DTranspose
from keras.layers import Input
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
import keras.backend as K
from keras.optimizers.legacy import Adam


class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        # foundation for 8x8 feature maps
        n_nodes = 1024*8*8
        model.add(Dense(n_nodes, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Reshape((8, 8, 1024)))
        # upsample to 16x16
        model.add(Conv2DTranspose(512, (4,4), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # upsample to 32x32
        model.add(Conv2DTranspose(256, (4,4), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # upsample to 64x64
        model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # upsample to 128x128
        model.add(Conv2DTranspose(64, (4,4), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # output layer 128x128x1
        model.add(Conv2D(1, (5,5), activation='sigmoid', padding='same'))

        model.summary()
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        model = Sequential()
        # normal
        model.add(Conv2D(64, (5,5), padding='same', input_shape=self.img_shape))
        model.add(LeakyReLU(alpha=0.2))
        # downsample to 64x64
        model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # downsample to 32x32
        model.add(Conv2D(256, (5,5), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # downsample to 16x16
        model.add(Conv2D(512, (5,5), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # downsample to 8x8
        model.add(Conv2D(1024, (5,5), strides=(2,2), padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        # classifier
        model.add(Flatten())
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50, last_model_point=0, summary_writer=None):

        # Rescale -1 to 1
        #X_train = X_train / 127.5 - 1.
        #X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        loss1, loss2, loss3, acc1, acc2 = [], [], [], [], []

        for epoch in range(epochs):


            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images

            idx = np.random.randint(0, x_train.shape[0], batch_size)
            imgs = np.array([x_train[i] for i in idx])

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real, d_acc_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake, d_acc_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add((d_loss_real,d_acc_real),(d_loss_fake,d_acc_fake))

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            #idx = np.random.randint(0, x_train.shape[0], batch_size)
            #imgs = np.array([x_train[i] for i in idx])
            #gen_loss, disc_loss = self.train_step(imgs, batch_size)
            #print ("%d [D loss: %f] [G loss: %f]" % (epoch, disc_loss, gen_loss))
            loss1.append(d_loss_real); loss2.append(d_loss_fake); loss3.append(g_loss); acc1.append(d_acc_real); acc2.append(d_acc_fake)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch, last_model_point, summary_writer['image'])
                self.generator.save('/content/drive/My Drive/Main/generated_models/Generator_model_{}'.format(epoch+last_model_point))
                self.discriminator.save('/content/drive/My Drive/Main/generated_models/Discriminator_model_{}'.format(epoch+last_model_point))

        loss = (loss1, loss2, loss3)
        acc = (acc1, acc2)

        if summary_writer['train_metrics']:
            with summary_writer['train_metrics'].as_default():
              # Log loss plot
              loss_plot_image = self.plot_loss(loss)
              tf.summary.image('Loss_Plot', loss_plot_image, step=0)

              # Log accuracy plot
              acc_plot_image = self.plot_acc(acc)
              tf.summary.image('Accuracy_Plot', acc_plot_image, step=0)

    '''
    def train_step(self, x_train, batch_size):
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            gen_loss, disc_loss = self.compute_loss(x_train, batch_size)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return gen_loss, disc_loss

    def compute_loss(self, real_images, batch_size):
        # Generate noise input for the generator
        noise = tf.random.normal([batch_size, self.latent_dim])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            # Generate fake images using the generator
            generated_images = self.generator(noise, training=True)

            # Get discriminator predictions for real and fake images
            real_output = self.discriminator(real_images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            # Calculate generator loss
            gen_loss = self.generator_loss(fake_output)

            # Calculate discriminator loss
            disc_loss = self.discriminator_loss(real_output, fake_output)

        return gen_loss, disc_loss

    def generator_loss(self, fake_output):
        # Generator loss: binary cross-entropy with target labels as valid (1)
        return tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output))

    def discriminator_loss(self, real_output, fake_output):
        # Discriminator loss: binary cross-entropy with target labels for real (1) and fake (0) images
        real_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.ones_like(real_output), real_output))
        fake_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output))
        return real_loss + fake_loss '''
    def plot_loss(self,loss):
        plt.figure(figsize=(10,5))
        plt.title("Generator and Discriminator Loss During Training", fontsize=20)
        plt.plot(loss[0], label="D_loss_real")
        plt.plot(loss[1], label="D_loss_fake")
        plt.plot(loss[2], label="G_loss")
        plt.xlabel("Iteration", fontsize=20); plt.ylabel("Loss", fontsize=20)
        plt.legend()
        plt.close()
        loss_plot_image = self.plt_to_image(plt)
        return loss_plot_image

    def plot_acc(self,acc):
        plt.figure(figsize=(10,5))
        plt.title("Generator and Discriminator accuracy During Training", fontsize=20)
        plt.plot(acc[0], label="D_acc_real")
        plt.plot(acc[1], label="D_acc_fake")
        plt.xlabel("Iteration", fontsize=20); plt.ylabel("accuracy", fontsize=20)
        plt.legend()
        plt.close()
        acc_plot_image = self.plt_to_image(plt)
        return acc_plot_image

    import io
    def plt_to_image(self,plt):
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        image = tf.expand_dims(image, 0)
        return image

    def save_imgs(self, epoch, last_model_point, summary_writer=None):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='bone')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/drive/My Drive/Main/generated_images/{}.png".format(epoch+last_model_point))
        plt.close()
        # Log selected images to TensorBoard
        if summary_writer:
            with summary_writer.as_default():
                tf.summary.image('Generated Image', gen_imgs, step=epoch+last_model_point, max_outputs=5)
            summary_writer.flush()

    '''def evaluate(self, x_test, batch_size, epoch=-1, summary_writer=None):
        d_loss_real_values = []
        d_loss_fake_values = []
        g_loss_values = []
        d_accuracy_real_values = []
        d_accuracy_fake_values = []

        num_batches = len(x_test) // batch_size

        for i in range(num_batches):
            # Get the current batch
            start_idx = i * batch_size
            end_idx = (i + 1) * batch_size
            batch_images = x_test[start_idx:end_idx]
            noise = np.random.normal(0, 1, (batch_images.shape[0], self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            d_loss_real, d_accuracy_real = self.discriminator.evaluate(batch_images, np.ones((batch_images.shape[0], 1)), verbose=0)
            d_loss_fake, d_accuracy_fake = self.discriminator.evaluate(gen_imgs, np.zeros((batch_images.shape[0], 1)), verbose=0)

            # Evaluate the combined model on noise and valid labels
            g_loss = self.combined.evaluate(noise, np.ones((batch_images.shape[0], 1)), verbose=0)

            d_loss_real_values.append(d_loss_real)
            d_loss_fake_values.append(d_loss_fake)
            g_loss_values.append(g_loss)
            d_accuracy_real_values.append(d_accuracy_real)
            d_accuracy_fake_values.append(d_accuracy_fake)

        #print(d_loss_real_values);print(d_loss_fake_values);print(g_loss_values);print(d_accuracy_real_values);print(d_accuracy_fake_values)

        # Log evaluation metrics to TensorBoard
        if summary_writer:
            with summary_writer.as_default():
                tf.summary.scalar('D Loss Real', np.mean(d_loss_real_values), step=epoch)
                tf.summary.scalar('D Loss Fake', np.mean(d_loss_fake_values), step=epoch)
                tf.summary.scalar('G Loss', np.mean(g_loss_values), step=epoch)
                tf.summary.scalar('D Accuracy Real', np.mean(d_accuracy_real_values), step=epoch)
                tf.summary.scalar('D Accuracy Fake', np.mean(d_accuracy_fake_values), step=epoch)
            summary_writer.flush()'''

from keras.models import load_model

def find_last_model_checkpoint():
    last_model_point=0
    for f in (glob('/content/drive/My Drive/Main/generated_models/Generator_model_*')):
        file = f.split('/')[-1]
        checkpoint_no = int(file.split('_')[-1])
        if checkpoint_no > last_model_point:
            last_model_point = checkpoint_no

    return int(last_model_point)

if __name__ == '__main__':
    dcgan = DCGAN()
    last_model_point=find_last_model_checkpoint()
    print ("Last checkpoint number = %d" %last_model_point)

    if os.path.exists('/content/drive/My Drive/Main/generated_models/Generator_model_{}'.format(last_model_point)):
        dcgan.generator = load_model('/content/drive/My Drive/Main/generated_models/Generator_model_{}'.format(last_model_point))
        dcgan.discriminator = load_model('/content/drive/My Drive/Main/generated_models/Discriminator_model_{}'.format(last_model_point))
        optimizer = Adam(0.0002, 0.5)
        dcgan.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        z = Input(shape=(100,))
        img = dcgan.generator(z)

        # For the combined model we will only train the generator
        dcgan.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = dcgan.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        dcgan.combined = Model(z, valid)
        dcgan.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    log_dir = '/content/drive/My Drive/Main/Tensorboard/dcgan/'
    summary_writer = {'image':tf.summary.create_file_writer(log_dir+'gen_images'),
                      'evaluation':tf.summary.create_file_writer(log_dir+'eval'),
                      'train_metrics':tf.summary.create_file_writer(log_dir+'train_metrics')}
    dcgan.train(epochs=4001, batch_size=128, save_interval=100,last_model_point=last_model_point, summary_writer=summary_writer)
    dcgan.evaluate(x_test, 128, summary_writer['evaluation'])
    summary_writer['image'].close()
    summary_writer['train_metrics'].close()
    summary_writer['evaluation'].close


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1664      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       204928    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 256)       819456    
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 32, 32, 256)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 512)       3

4/4 [==============================] - 0s 90ms/step
1 [D loss: 0.727502, acc.: 0.00%] [G loss: 0.697382]
4/4 [==============================] - 0s 57ms/step
2 [D loss: 0.692612, acc.: 48.44%] [G loss: 0.697476]
4/4 [==============================] - 0s 58ms/step
3 [D loss: 0.690040, acc.: 50.78%] [G loss: 0.701875]
4/4 [==============================] - 0s 55ms/step
4 [D loss: 0.686281, acc.: 55.08%] [G loss: 0.716854]
4/4 [==============================] - 0s 56ms/step
5 [D loss: 0.680562, acc.: 59.38%] [G loss: 0.755784]
4/4 [==============================] - 0s 56ms/step
6 [D loss: 0.666815, acc.: 82.03%] [G loss: 0.913003]
4/4 [==============================] - 0s 57ms/step
7 [D loss: 0.654545, acc.: 50.78%] [G loss: 1.315535]
4/4 [==============================] - 0s 56ms/step
8 [D loss: 0.706580, acc.: 30.47%] [G loss: 1.367444]
4/4 [==============================] - 0s 57ms/step
9 [D loss: 0.542261, acc.: 83.20%] [G loss: 2.002287]
4/4 [==============================] - 0s 58ms/

4/4 [==============================] - 0s 89ms/step
101 [D loss: 0.658493, acc.: 62.50%] [G loss: 0.881579]
4/4 [==============================] - 0s 57ms/step
102 [D loss: 0.605647, acc.: 75.00%] [G loss: 0.920030]
4/4 [==============================] - 0s 60ms/step
103 [D loss: 0.586232, acc.: 78.12%] [G loss: 0.983249]
4/4 [==============================] - 0s 58ms/step
104 [D loss: 0.562725, acc.: 78.12%] [G loss: 1.004447]
4/4 [==============================] - 0s 59ms/step
105 [D loss: 0.523869, acc.: 80.08%] [G loss: 1.122955]
4/4 [==============================] - 0s 60ms/step
106 [D loss: 0.531241, acc.: 76.56%] [G loss: 1.167370]
4/4 [==============================] - 0s 59ms/step
107 [D loss: 0.551621, acc.: 73.83%] [G loss: 1.120980]
4/4 [==============================] - 0s 60ms/step
108 [D loss: 0.575813, acc.: 66.02%] [G loss: 1.101672]
4/4 [==============================] - 0s 59ms/step
109 [D loss: 0.637170, acc.: 62.89%] [G loss: 0.966011]
4/4 [=======================

4/4 [==============================] - 0s 78ms/step
201 [D loss: 0.566435, acc.: 72.27%] [G loss: 1.223583]
4/4 [==============================] - 0s 57ms/step
202 [D loss: 0.556717, acc.: 77.34%] [G loss: 1.280963]
4/4 [==============================] - 0s 58ms/step
203 [D loss: 0.597325, acc.: 69.53%] [G loss: 1.209785]
4/4 [==============================] - 0s 59ms/step
204 [D loss: 0.639268, acc.: 61.72%] [G loss: 1.062510]
4/4 [==============================] - 0s 59ms/step
205 [D loss: 0.550409, acc.: 73.83%] [G loss: 1.198374]
4/4 [==============================] - 0s 58ms/step
206 [D loss: 0.603312, acc.: 68.36%] [G loss: 1.168793]
4/4 [==============================] - 0s 59ms/step
207 [D loss: 0.595292, acc.: 69.14%] [G loss: 1.259799]
4/4 [==============================] - 0s 59ms/step
208 [D loss: 0.597671, acc.: 70.70%] [G loss: 1.340578]
4/4 [==============================] - 0s 60ms/step
209 [D loss: 0.563862, acc.: 76.56%] [G loss: 1.443077]
4/4 [=======================

4/4 [==============================] - 0s 84ms/step
301 [D loss: 0.553913, acc.: 71.48%] [G loss: 1.708968]
4/4 [==============================] - 0s 58ms/step
302 [D loss: 0.463520, acc.: 82.81%] [G loss: 1.959890]
4/4 [==============================] - 0s 57ms/step
303 [D loss: 0.485843, acc.: 80.47%] [G loss: 1.846304]
4/4 [==============================] - 0s 58ms/step
304 [D loss: 0.511996, acc.: 74.61%] [G loss: 1.969996]
4/4 [==============================] - 0s 58ms/step
305 [D loss: 0.550788, acc.: 73.05%] [G loss: 1.939146]
4/4 [==============================] - 0s 60ms/step
306 [D loss: 0.566598, acc.: 69.14%] [G loss: 1.653661]
4/4 [==============================] - 0s 58ms/step
307 [D loss: 0.503756, acc.: 76.56%] [G loss: 1.758907]
4/4 [==============================] - 0s 60ms/step
308 [D loss: 0.493946, acc.: 79.30%] [G loss: 2.255359]
4/4 [==============================] - 0s 58ms/step
309 [D loss: 0.575532, acc.: 68.36%] [G loss: 1.701223]
4/4 [=======================

...

4/4 [==============================] - 0s 78ms/step
2701 [D loss: 0.187235, acc.: 98.83%] [G loss: 74.806656]
4/4 [==============================] - 0s 57ms/step
2702 [D loss: 0.002903, acc.: 99.61%] [G loss: 73.454941]
4/4 [==============================] - 0s 57ms/step
2703 [D loss: 0.432474, acc.: 98.83%] [G loss: 70.397400]
4/4 [==============================] - 0s 58ms/step
2704 [D loss: 0.094484, acc.: 98.83%] [G loss: 71.639297]
4/4 [==============================] - 0s 60ms/step
2705 [D loss: 0.099496, acc.: 99.22%] [G loss: 66.117874]
4/4 [==============================] - 0s 60ms/step
2706 [D loss: 0.035600, acc.: 99.61%] [G loss: 68.276718]
4/4 [==============================] - 0s 59ms/step
2707 [D loss: 0.013498, acc.: 99.61%] [G loss: 63.850491]
4/4 [==============================] - 0s 60ms/step
2708 [D loss: 0.036634, acc.: 99.22%] [G loss: 69.242813]
4/4 [==============================] - 0s 60ms/step
2709 [D loss: 0.001430, acc.: 100.00%] [G loss: 72.422256]
4/4 [====

4/4 [==============================] - 0s 91ms/step
2801 [D loss: 0.244132, acc.: 98.83%] [G loss: 71.421745]
4/4 [==============================] - 0s 59ms/step
2802 [D loss: 0.000000, acc.: 100.00%] [G loss: 71.892227]
4/4 [==============================] - 0s 59ms/step
2803 [D loss: 0.013311, acc.: 99.61%] [G loss: 73.127205]
4/4 [==============================] - 0s 59ms/step
2804 [D loss: 0.000000, acc.: 100.00%] [G loss: 73.477570]
4/4 [==============================] - 0s 59ms/step
2805 [D loss: 0.021660, acc.: 99.61%] [G loss: 66.278152]
4/4 [==============================] - 0s 54ms/step
2806 [D loss: 0.149397, acc.: 99.22%] [G loss: 64.776230]
4/4 [==============================] - 0s 59ms/step
2807 [D loss: 0.000000, acc.: 100.00%] [G loss: 66.389091]
4/4 [==============================] - 0s 59ms/step
2808 [D loss: 0.000000, acc.: 100.00%] [G loss: 67.990051]
4/4 [==============================] - 0s 59ms/step
2809 [D loss: 0.080701, acc.: 99.22%] [G loss: 63.786640]
4/4 [=

4/4 [==============================] - 0s 79ms/step
2901 [D loss: 0.000013, acc.: 100.00%] [G loss: 63.599812]
4/4 [==============================] - 0s 58ms/step
2902 [D loss: 0.078998, acc.: 99.61%] [G loss: 61.345390]
4/4 [==============================] - 0s 58ms/step
2903 [D loss: 0.000010, acc.: 100.00%] [G loss: 63.492233]
4/4 [==============================] - 0s 59ms/step
2904 [D loss: 0.000001, acc.: 100.00%] [G loss: 61.441536]
4/4 [==============================] - 0s 59ms/step
2905 [D loss: 0.000001, acc.: 100.00%] [G loss: 60.150154]
4/4 [==============================] - 0s 59ms/step
2906 [D loss: 0.014070, acc.: 99.61%] [G loss: 61.978718]
4/4 [==============================] - 0s 60ms/step
2907 [D loss: 0.057367, acc.: 99.22%] [G loss: 62.346302]
4/4 [==============================] - 0s 61ms/step
2908 [D loss: 0.116147, acc.: 99.22%] [G loss: 61.445278]
4/4 [==============================] - 0s 59ms/step
2909 [D loss: 0.002591, acc.: 100.00%] [G loss: 62.670509]
4/4 [

4/4 [==============================] - 0s 80ms/step
3001 [D loss: 0.000055, acc.: 100.00%] [G loss: 70.608109]
4/4 [==============================] - 0s 60ms/step
3002 [D loss: 0.000000, acc.: 100.00%] [G loss: 72.115219]
4/4 [==============================] - 0s 58ms/step
3003 [D loss: 0.000037, acc.: 100.00%] [G loss: 70.413322]
4/4 [==============================] - 0s 59ms/step
3004 [D loss: 0.000000, acc.: 100.00%] [G loss: 69.720978]
4/4 [==============================] - 0s 59ms/step
3005 [D loss: 0.478585, acc.: 99.61%] [G loss: 67.911514]
4/4 [==============================] - 0s 60ms/step
3006 [D loss: 0.000693, acc.: 100.00%] [G loss: 63.590385]
4/4 [==============================] - 0s 60ms/step
3007 [D loss: 0.000000, acc.: 100.00%] [G loss: 63.713108]
4/4 [==============================] - 0s 61ms/step
3008 [D loss: 0.011930, acc.: 99.61%] [G loss: 67.349106]
4/4 [==============================] - 0s 61ms/step
3009 [D loss: 0.004247, acc.: 99.61%] [G loss: 72.456528]
4/4 

4/4 [==============================] - 0s 79ms/step
3101 [D loss: 0.000000, acc.: 100.00%] [G loss: 66.729218]
4/4 [==============================] - 0s 57ms/step
3102 [D loss: 0.016354, acc.: 99.61%] [G loss: 62.356873]
4/4 [==============================] - 0s 57ms/step
3103 [D loss: 0.015526, acc.: 99.61%] [G loss: 62.404488]
4/4 [==============================] - 0s 59ms/step
3104 [D loss: 0.000297, acc.: 100.00%] [G loss: 61.270161]
4/4 [==============================] - 0s 60ms/step
3105 [D loss: 0.009069, acc.: 99.61%] [G loss: 61.504051]
4/4 [==============================] - 0s 60ms/step
3106 [D loss: 0.000000, acc.: 100.00%] [G loss: 66.445068]
4/4 [==============================] - 0s 60ms/step
3107 [D loss: 0.000000, acc.: 100.00%] [G loss: 63.464069]
4/4 [==============================] - 0s 60ms/step
3108 [D loss: 0.093065, acc.: 99.22%] [G loss: 61.486198]
4/4 [==============================] - 0s 61ms/step
3109 [D loss: 0.000000, acc.: 100.00%] [G loss: 59.081219]
4/4 [